In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier

# Train_data

In [2]:
df = pd.read_csv("school-21-student-expulsion-prediction/train.csv")

In [3]:
to_drop = ['id','Heard about school from','Contract termination date']
to_drop_test = ['Heard about school from','Contract termination date']

In [4]:
native_city = list(map(str, list(df['Native city'].unique())))

native_city = [i.lower() for i in native_city]

In [5]:
df = df.drop(to_drop, axis=1)

Оставим в Native city Москву и все похожие названия,также города Московской области

In [6]:
moscow = []

for i in native_city:
    if "москв" in i or "mos" in i or 'балаш' in i or 'видное' in i or 'красногорск' in i or 'люберцы' in i or 'подольск'in i or 'химки' in i or 'мытищи' in i or 'щербинка' in i:
        moscow.append(i)

In [7]:
moscow

['moscow',
 'москва',
 'химки',
 'гор. мытищи',
 'москва ',
 'балашиха',
 'мытищи',
 'москве',
 'подольск',
 'красногорск',
 ' москва',
 'люберцы',
 'г. москва',
 'москвп',
 'mosocow',
 'г. москва, ',
 'балашиха, мкр. железнодорожный',
 'moscow region, odintsovo',
 'видное',
 'московская обл. г. мытищи',
 'гор. балашиха ',
 'город москва',
 'мытищи ',
 'г. химки']

In [8]:
df['Native city']= df['Native city'].str.lower()

In [9]:
for i in np.arange(df.shape[0]):
    if df['Native city'][i] in moscow:
        df['Native city'][i] = 1
    else:
        df['Native city'][i] = 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df['Native city'].unique()

array([0, 1], dtype=object)

Преобразовываем дату рождения в возраст

In [11]:
import datetime as dt

In [12]:
now = pd.to_datetime('now')

In [13]:
df['Birth date'] = pd.to_datetime(df['Birth date'], errors='coerce')

In [14]:
df['Birth date'] = (now.year - df['Birth date'].dt.year) - ((now.month - df['Birth date'].dt.month) < 0)

In [15]:
df

,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,0,male,3,7.463235,work,5,NaN,0.0,80.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,female,1,8.785714,NaN,0,NaN,10.0,45.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,female,2,6.775000,work,0,NaN,20.0,2.0,...,NaN,NaN,NaN,0.0,0.0,16.0,27.0,7,11,1
3,23,0,female,1,8.146104,NaN,5,0.0,35.0,25.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,0,male,3,4.272727,other,0,0.0,NaN,5.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,0,male,3,4.149733,study,10,0.0,0.0,10.0,...,NaN,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,0,male,3,4.320856,study,5,0.0,10.0,0.0,...,120.0,0.0,NaN,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,male,2,6.675000,study,30,30.0,10.0,65.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,male,3,6.483333,NaN,50,NaN,80.0,30.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [17]:
df['Gender'].replace({'female':0,'male':1},inplace=True)

In [18]:
df

,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,0,1,3,7.463235,work,5,NaN,0.0,80.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,0,1,8.785714,NaN,0,NaN,10.0,45.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,0,2,6.775000,work,0,NaN,20.0,2.0,...,NaN,NaN,NaN,0.0,0.0,16.0,27.0,7,11,1
3,23,0,0,1,8.146104,NaN,5,0.0,35.0,25.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,0,1,3,4.272727,other,0,0.0,NaN,5.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,0,1,3,4.149733,study,10,0.0,0.0,10.0,...,NaN,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,0,1,3,4.320856,study,5,0.0,10.0,0.0,...,120.0,0.0,NaN,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,1,2,6.675000,study,30,30.0,10.0,65.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,1,3,6.483333,NaN,50,NaN,80.0,30.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


Life status разобъем на две категории: work и study = 0, а other и NaN = 1, так как вероятность подписания договора незанятого человека выше 

In [19]:
df['Life status'].replace({'work':0,'study':0,'other':1},inplace=True)


In [20]:
df

,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,0,1,3,7.463235,0.0,5,NaN,0.0,80.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,0,1,8.785714,NaN,0,NaN,10.0,45.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,0,2,6.775000,0.0,0,NaN,20.0,2.0,...,NaN,NaN,NaN,0.0,0.0,16.0,27.0,7,11,1
3,23,0,0,1,8.146104,NaN,5,0.0,35.0,25.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,0,1,3,4.272727,1.0,0,0.0,NaN,5.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,0,1,3,4.149733,0.0,10,0.0,0.0,10.0,...,NaN,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,0,1,3,4.320856,0.0,5,0.0,10.0,0.0,...,120.0,0.0,NaN,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,1,2,6.675000,0.0,30,30.0,10.0,65.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,1,3,6.483333,NaN,50,NaN,80.0,30.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [21]:
df['Life status']=df['Life status'].fillna(1)

In [22]:
df = df.fillna(0)

In [23]:
df

,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,0,1,3,7.463235,0.0,5,0.0,0.0,80.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,0,1,8.785714,1.0,0,0.0,10.0,45.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,0,2,6.775000,0.0,0,0.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,11,1
3,23,0,0,1,8.146104,1.0,5,0.0,35.0,25.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,0,1,3,4.272727,1.0,0,0.0,0.0,5.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,0,1,3,4.149733,0.0,10,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,0,1,3,4.320856,0.0,5,0.0,10.0,0.0,...,120.0,0.0,0.0,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,1,2,6.675000,0.0,30,30.0,10.0,65.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,1,3,6.483333,1.0,50,0.0,80.0,30.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [24]:
df.to_csv('data/train_after_clean.csv', index=False)# cохраяняем тренировочные данные

# Correlation

In [25]:
corr = df.corr()

In [26]:
corr.to_excel("data/correlation.xlsx")

Очистим также и тестовые данные

# Test_data

In [27]:
test_data = pd.read_csv("school-21-student-expulsion-prediction/test.csv")

In [28]:
test_data['Native city']= test_data['Native city'].str.lower()

In [29]:
for i in np.arange(test_data.shape[0]):
    if test_data['Native city'][i] in moscow:
        test_data['Native city'][i] = 1
    else:
        test_data['Native city'][i] = 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
id_student = test_data['id']

In [31]:
test_data = test_data.drop(to_drop_test, axis=1)

In [32]:
test_data['Gender'].replace({'female':0,'male':1},inplace=True)

In [33]:
now = pd.to_datetime('now')
test_data['Birth date'] = pd.to_datetime(test_data['Birth date'], errors='coerce')
test_data['Birth date'] = (now.year - test_data['Birth date'].dt.year) - ((now.month - test_data['Birth date'].dt.month) < 0)

In [34]:
test_data

,id,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,...,bsq,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game
0,685512434,27,0,1,3,8.834416,work,10,NaN,80.0,...,100.0,120.0,0.0,0.0,75.0,43.0,64.0,45.0,10,12
1,157712083,26,1,1,2,2.172414,work,7,0.0,0.0,...,0.0,0.0,0.0,NaN,80.0,22.0,33.0,36.0,7,6
2,672158583,31,1,1,3,4.310160,other,5,0.0,0.0,...,0.0,0.0,0.0,75.0,0.0,6.0,33.0,9.0,7,10
3,1205321261,22,0,0,2,6.295833,study,20,0.0,30.0,...,0.0,100.0,0.0,0.0,15.0,1.0,16.0,22.0,9,8
4,621191476,23,1,1,1,7.110294,NaN,100,100.0,80.0,...,100.0,0.0,0.0,0.0,65.0,65.0,54.0,36.0,9,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,1040277160,23,0,1,3,5.726415,NaN,0,30.0,35.0,...,100.0,120.0,0.0,0.0,100.0,54.0,32.0,30.0,8,11
702,1273857893,21,1,1,4,2.475862,study,40,55.0,0.0,...,0.0,NaN,0.0,0.0,0.0,27.0,27.0,22.0,7,12
703,1465130708,26,1,1,1,8.724026,NaN,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,7
704,1404312854,31,0,1,4,6.766667,work,32,NaN,0.0,...,0.0,120.0,0.0,0.0,0.0,0.0,48.0,31.0,8,7


In [35]:
test_data['Life status'].replace({'work':0,'study':0,'other':1},inplace=True)

In [36]:
test_data['Life status']=test_data['Life status'].fillna(1)

In [37]:
for i in np.arange(test_data.shape[1]):
  test_data.iloc[:, i].fillna(0, inplace=True)

In [38]:
test_data.to_csv('data/test_data_after_clean.csv', index=False)

In [42]:
df#Тренировные данные


,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,0,1,3,7.463235,0.0,5,0.0,0.0,80.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,0,1,8.785714,1.0,0,0.0,10.0,45.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,0,2,6.775000,0.0,0,0.0,20.0,2.0,...,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,11,1
3,23,0,0,1,8.146104,1.0,5,0.0,35.0,25.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,0,1,3,4.272727,1.0,0,0.0,0.0,5.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,0,1,3,4.149733,0.0,10,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,0,1,3,4.320856,0.0,5,0.0,10.0,0.0,...,120.0,0.0,0.0,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,1,2,6.675000,0.0,30,30.0,10.0,65.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,1,3,6.483333,1.0,50,0.0,80.0,30.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [43]:
test_data#тестовые данные

,id,Birth date,Native city,Gender,Wave id,Level,Life status,day_00,day_01,day_02,...,bsq,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game
0,685512434,27,0,1,3,8.834416,0.0,10,0.0,80.0,...,100.0,120.0,0.0,0.0,75.0,43.0,64.0,45.0,10,12
1,157712083,26,1,1,2,2.172414,0.0,7,0.0,0.0,...,0.0,0.0,0.0,0.0,80.0,22.0,33.0,36.0,7,6
2,672158583,31,1,1,3,4.310160,1.0,5,0.0,0.0,...,0.0,0.0,0.0,75.0,0.0,6.0,33.0,9.0,7,10
3,1205321261,22,0,0,2,6.295833,0.0,20,0.0,30.0,...,0.0,100.0,0.0,0.0,15.0,1.0,16.0,22.0,9,8
4,621191476,23,1,1,1,7.110294,1.0,100,100.0,80.0,...,100.0,0.0,0.0,0.0,65.0,65.0,54.0,36.0,9,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,1040277160,23,0,1,3,5.726415,1.0,0,30.0,35.0,...,100.0,120.0,0.0,0.0,100.0,54.0,32.0,30.0,8,11
702,1273857893,21,1,1,4,2.475862,0.0,40,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,22.0,7,12
703,1465130708,26,1,1,1,8.724026,1.0,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,7
704,1404312854,31,0,1,4,6.766667,0.0,32,0.0,0.0,...,0.0,120.0,0.0,0.0,0.0,0.0,48.0,31.0,8,7
